In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the tickers and download the data
tickers = ["ALL", "ASGN", "CI", "COP", "EME", "EVR", "GILD", "GPK", 
           "ISRG", "MKL", "MOH", "PEG", "PXD", "QCOM", "UBSI", "VFC", "XEL"]

# Download the adjusted close prices for the specified time period
start_date = "2017-11-27"
end_date = "2019-11-27"
data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

# Initialize dataframes for results
ror = pd.DataFrame(index=tickers, columns=['BandH', 'MACD', 'RSI'])
risk = pd.DataFrame(index=tickers, columns=['BandH', 'MACD', 'RSI'])
sharpe_ratio = pd.DataFrame(index=tickers, columns=['BandH', 'MACD', 'RSI'])

for ticker in tickers:
    # Calculate MACD and Signal Line
    data['MACD'] = data[ticker].ewm(span=12, adjust=False).mean() - data[ticker].ewm(span=26, adjust=False).mean()
    data['Signal_Line'] = data['MACD'].ewm(span=9, adjust=False).mean()

    # Calculate RSI
    delta = data[ticker].diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    data['RSI'] = 100 - (100 / (1 + rs))

    # Buy and Hold
    data['BandH_Returns'] = data[ticker].pct_change().fillna(0)

    # MACD
    data['MACD_Returns'] = np.where(data['MACD'] > data['Signal_Line'], data[ticker].pct_change(), 0)

    # RSI
    data['RSI_Returns'] = np.where(data['RSI'] < 30, data[ticker].pct_change(), 
                                   np.where(data['RSI'] > 70, -data[ticker].pct_change(), 0))

    # Store metrics
    ror.loc[ticker] = [data['BandH_Returns'].sum(), data['MACD_Returns'].sum(), data['RSI_Returns'].sum()]
    risk.loc[ticker] = [data['BandH_Returns'].std(), data['MACD_Returns'].std(), data['RSI_Returns'].std()]
    sharpe_ratio.loc[ticker] = ror.loc[ticker] - 0.025 / risk.loc[ticker]

# Print results
print("Rate of Return:\n", ror)
print("\nRisk:\n", risk)
print("\nSharpe Ratio:\n", sharpe_ratio)


[*********************100%%**********************]  17 of 17 completed
Rate of Return:
          BandH      MACD       RSI
ALL   0.199301  0.498368 -0.595124
ASGN  0.161419  1.132448 -1.455774
CI    0.084516   0.63507 -0.939568
COP   0.323553  0.963891 -1.010666
EME   0.206767  0.756185 -0.867812
EVR   0.054432   0.75576 -1.172999
GILD  0.064041  0.494844 -0.532874
GPK   0.193111  0.815494 -1.027604
ISRG  0.470455   0.87648 -0.894701
MKL    0.08292  0.314591 -0.436752
MOH   0.721575  1.323886  -1.68478
PEG   0.236139  0.511066 -0.393902
PXD  -0.042431  0.904586 -1.119371
QCOM  0.411709  1.318585 -1.979384
UBSI  0.212503  0.677339  -0.87672
VFC   0.353037  0.838533 -0.928891
XEL   0.272316  0.506921 -0.684663

Risk:
          BandH      MACD       RSI
ALL   0.011881  0.007019  0.005915
ASGN   0.02024  0.014647  0.011356
CI    0.017965  0.012126   0.01048
COP   0.018122  0.012117  0.009673
EME   0.014503  0.009084  0.007389
EVR   0.018372  0.012641   0.01035
GILD   0.01557  0.009764  0.0

In [2]:
print(ror["BandH"].mean())
print(ror["RSI"].mean())
print(ror["MACD"].mean())

0.23560957390693987
-0.9765636727026779
0.7837675171738485


In [3]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the tickers and download the data
tickers = ["ALL", "ASGN", "CI", "COP", "EME", "EVR", "GILD", "GPK", 
           "ISRG", "MKL", "MOH", "PEG", "PXD", "QCOM", "UBSI", "VFC", "XEL"]

# Download the adjusted close prices for the specified time period
start_date = "2017-11-27"
end_date = "2019-11-27"
data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

# MACD strategy
def compute_macd_strategy(data):
    short_window = 12
    long_window = 26
    signal_window = 9
    
    data['12_ema'] = data['Adj Close'].ewm(span=short_window, adjust=False).mean()
    data['26_ema'] = data['Adj Close'].ewm(span=long_window, adjust=False).mean()
    data['macd'] = data['12_ema'] - data['26_ema']
    data['signal'] = data['macd'].ewm(span=signal_window, adjust=False).mean()
    
    data['macd_signal'] = np.where(data['macd'] > data['signal'], 1, -1)
    
    return data

# RSI strategy
def compute_rsi_strategy(data):
    rsi_period = 14
    
    delta = data['Adj Close'].diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    avg_gain = gain.rolling(window=rsi_period).mean()
    avg_loss = loss.rolling(window=rsi_period).mean()
    rs = avg_gain / avg_loss
    data['rsi'] = 100 - (100 / (1 + rs))
    
    data['rsi_signal'] = np.where(data['rsi'] < 30, 1, np.where(data['rsi'] > 70, -1, 0))
    
    return data

# Buy and Hold strategy
def compute_bandh_strategy(data):
    data['bandh_signal'] = 1
    return data


# Calculate Rate of Return, Risk, and Sharpe Ratio
def compute_performance_metrics(data, strategy_column, risk_free_rate=2.5):
    transaction_cost = 0.00025

    data['position'] = data[strategy_column].shift(1)
    data['entry'] = np.where(data['position'] == 1, data['Adj Close'], np.nan)
    data['exit'] = np.where(data['position'] == -1, data['Adj Close'], np.nan)
    data['entry'].ffill(inplace=True)
    data['returns'] = np.where(data['position'] == -1, 
                               (data['exit'] - (data['entry'] * (1 + transaction_cost))) / data['entry'], 
                               np.nan)
    returns = np.array(data['returns'])
    returns_only = returns[~np.isnan(returns)]

    rate_of_return = sum(returns_only)
    risk = returns_only.std()
    sharpe_ratio = (rate_of_return - risk_free_rate) / risk
    
    return rate_of_return, risk, sharpe_ratio

# Apply strategies and compute metrics
for ticker in tickers:
    df = data[[ticker]].rename(columns={ticker: 'Adj Close'})
    
    df_macd = compute_macd_strategy(df.copy())
    df_rsi = compute_rsi_strategy(df.copy())
    df_bandh = compute_bandh_strategy(df.copy())
    
    macd_return, macd_risk, macd_sharpe = compute_performance_metrics(df_macd, 'macd_signal')
    rsi_return, rsi_risk, rsi_sharpe = compute_performance_metrics(df_rsi, 'rsi_signal')
    bandh_return, bandh_risk, bandh_sharpe = compute_performance_metrics(df_bandh, 'bandh_signal')
    
    print(f"Metrics for {ticker}:")
    print(f"MACD - Rate of Return: {macd_return:.2f}, Risk: {macd_risk:.2f}, Sharpe Ratio: {macd_sharpe:.2f}")
    print(f"RSI - Rate of Return: {rsi_return:.2f}, Risk: {rsi_risk:.2f}, Sharpe Ratio: {rsi_sharpe:.2f}")
    print(f"Buy and Hold - Rate of Return: {bandh_return:.2f}, Risk: {bandh_risk:.2f}, Sharpe Ratio: {bandh_sharpe:.2f}")
    print("-" * 50)


[*********************100%%**********************]  17 of 17 completed
Metrics for ALL:
MACD - Rate of Return: -1.77, Risk: 0.03, Sharpe Ratio: -165.00
RSI - Rate of Return: 12.95, Risk: 0.12, Sharpe Ratio: 84.52
Buy and Hold - Rate of Return: 0.00, Risk: nan, Sharpe Ratio: nan
--------------------------------------------------
Metrics for ASGN:
MACD - Rate of Return: -11.20, Risk: 0.08, Sharpe Ratio: -168.68
RSI - Rate of Return: 7.39, Risk: 0.05, Sharpe Ratio: 99.72
Buy and Hold - Rate of Return: 0.00, Risk: nan, Sharpe Ratio: nan
--------------------------------------------------
Metrics for CI:
MACD - Rate of Return: -4.97, Risk: 0.04, Sharpe Ratio: -169.98
RSI - Rate of Return: 11.62, Risk: 0.09, Sharpe Ratio: 106.05
Buy and Hold - Rate of Return: 0.00, Risk: nan, Sharpe Ratio: nan
--------------------------------------------------
Metrics for COP:
MACD - Rate of Return: -7.12, Risk: 0.04, Sharpe Ratio: -233.12
RSI - Rate of Return: 10.85, Risk: 0.13, Sharpe Ratio: 66.75
Buy and H

/Users/sheehabpranto/projects/Dissertation/Code/.env/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/sheehabpranto/projects/Dissertation/Code/.env/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/sheehabpranto/projects/Dissertation/Code/.env/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/sheehabpranto/projects/Dissertation/Code/.env/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/sheehabpranto/projects/Dissertation/Code/.env/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value enc

Metrics for MOH:
MACD - Rate of Return: -10.30, Risk: 0.07, Sharpe Ratio: -176.87
RSI - Rate of Return: 42.49, Risk: 0.32, Sharpe Ratio: 124.27
Buy and Hold - Rate of Return: 0.00, Risk: nan, Sharpe Ratio: nan
--------------------------------------------------
Metrics for PEG:
MACD - Rate of Return: -2.66, Risk: 0.02, Sharpe Ratio: -251.34
RSI - Rate of Return: 6.62, Risk: 0.05, Sharpe Ratio: 82.50
Buy and Hold - Rate of Return: 0.00, Risk: nan, Sharpe Ratio: nan
--------------------------------------------------
Metrics for PXD:
MACD - Rate of Return: -11.40, Risk: 0.05, Sharpe Ratio: -253.12
RSI - Rate of Return: 6.46, Risk: 0.08, Sharpe Ratio: 50.71
Buy and Hold - Rate of Return: 0.00, Risk: nan, Sharpe Ratio: nan
--------------------------------------------------
Metrics for QCOM:
MACD - Rate of Return: -10.89, Risk: 0.06, Sharpe Ratio: -217.58
RSI - Rate of Return: 24.47, Risk: 0.20, Sharpe Ratio: 110.38
Buy and Hold - Rate of Return: 0.00, Risk: nan, Sharpe Ratio: nan
-----------

/Users/sheehabpranto/projects/Dissertation/Code/.env/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/sheehabpranto/projects/Dissertation/Code/.env/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/sheehabpranto/projects/Dissertation/Code/.env/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/sheehabpranto/projects/Dissertation/Code/.env/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/sheehabpranto/projects/Dissertation/Code/.env/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value enc

In [4]:
import pandas as pd

def compute_rsi(data, window):
    """
    Compute the RSI (Relative Strength Index) of a pandas DataFrame.
    
    Parameters:
    - data (pd.Series): A pandas Series containing the price data.
    - window (int): The period for which RSI should be calculated.
    
    Returns:
    - pd.Series: A pandas Series containing the RSI values.
    """
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)

    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

# Sample data
data = pd.Series([44.34, 44.09, 44.15, 43.61, 44.33, 44.83, 45.10, 45.42, 
                  45.84, 46.08, 45.89, 46.03, 45.61, 46.28, 46.28, 46.00, 
                  46.03, 46.41, 46.22, 46.29, 46.53, 46.48, 46.46, 46.23, 
                  46.08, 46.03, 45.61, 45.00, 45.03, 44.23, 43.95, 43.63])

rsi = compute_rsi(data, 14)
print(rsi)

0           NaN
1      0.000000
2     19.354839
3      7.058824
4     49.681529
5     61.835749
6     66.239316
7     70.300752
8     74.350649
9     76.204819
10    72.079772
11    73.150685
12    65.601966
13    70.464135
14    70.464135
15    70.020964
16    69.831224
17    80.567686
18    73.333333
19    70.165746
20    69.916435
21    65.963855
22    60.616438
23    52.577320
24    53.310105
25    50.000000
26    50.000000
27    26.470588
28    27.272727
29    22.935780
30    20.454545
31     9.826590
dtype: float64


In [6]:
import pandas as pd
import numpy as np
import ta
import yfinance as yf

def compute_rsi(data, window):
    """
    Compute the RSI (Relative Strength Index) of a pandas DataFrame.
    
    Parameters:
    - data (pd.Series): A pandas Series containing the price data.
    - window (int): The period for which RSI should be calculated.
    
    Returns:
    - pd.Series: A pandas Series containing the RSI values.
    """
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)

    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

def compute_macd(data, short_window=12, long_window=26, signal_window=9):
    """
    Compute the MACD (Moving Average Convergence Divergence) of a pandas DataFrame.
    
    Parameters:
    - data (pd.Series): A pandas Series containing the price data.
    - short_window (int): The short period for EMA. Default is 12.
    - long_window (int): The long period for EMA. Default is 26.
    - signal_window (int): The signal period for EMA. Default is 9.
    
    Returns:
    - pd.DataFrame: A pandas DataFrame containing the MACD and Signal values.
    """
    short_ema = data.ewm(span=short_window, adjust=False).mean()
    long_ema = data.ewm(span=long_window, adjust=False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()

    return pd.DataFrame({'MACD': macd, 'Signal': signal})


def calculate_macd_matrices(prices):
    """
    Calculate MACD and Signal Line indicators.
    
    Parameters:
    - prices (pd.Series): Series of prices.
    
    Returns:
    - Tuple[float, float]: MACD, Signal Line.
    """

    macd = compute_macd(prices, short_window=12, long_window=26, signal_window=9)

    # ta.trend.MACD(prices, window_fast=12, window_slow=26, window_sign=9)
    # macd, macd_signals = macd_series.macd(), macd_series.macd_signal()
    macd_returns = []
    buy_price = None

    for i in range(1, prices.shape[0]):
        if macd["MACD"].iloc[i] > macd["Signal"].iloc[i] and macd["MACD"].iloc[i - 1] <= macd["Signal"].iloc[i - 1]:
            buy_price = prices.iloc[i]
        elif macd["MACD"].iloc[i] < macd["Signal"].iloc[i] and macd["MACD"].iloc[i-1] >= macd["Signal"].iloc[i-1] and buy_price is not None:
            macd_returns.append((prices.iloc[i] - (buy_price * 1.00025) ) / buy_price)
            buy_price = 0
    macd_returns = np.array(macd_returns)
    try:
        if macd_returns.std() == 0:
            macd_sharpe_ratio = 0
        else:
            macd_sharpe_ratio = (macd_returns.sum() - 0.025) / macd_returns.std()
    except Exception as e:
        macd_sharpe_ratio = 0
    return macd_returns.sum(), macd_returns.std(), macd_sharpe_ratio

def calculate_rsi_matrices(prices):
    """
    Calculate RSI indicator.
    
    Parameters:
    - prices (pd.Series): Series of prices.
    
    Returns:
    - Tuple[float, float]: RSI, RSI Signal.
    """
    rsi_series = compute_rsi(prices, 14)
    rsi = rsi_series

    rsi_returns = []
    buy_price = None

    for i in range(prices.shape[0]):
        if rsi.iloc[i] < 30:
            buy_price = prices.iloc[i]
        elif rsi.iloc[i] > 70 and buy_price is not None:
            rsi_returns.append((prices.iloc[i] - (buy_price * 1.00025)) / buy_price)
            buy_price = None

    rsi_returns = np.array(rsi_returns)
    try:
        if rsi_returns.std() == 0:
            rsi_sharpe_ratio = 0
        else:
            rsi_sharpe_ratio = (rsi_returns.mean() - 0.025) / rsi_returns.std()
    except Exception as e:
        rsi_sharpe_ratio = 0
    return rsi_returns.mean(), rsi_returns.std(), rsi_sharpe_ratio

def calculate_bandh_matrices(prices):
    """
    Calculate Buy and Hold indicator.
    
    Parameters:
    - prices (pd.Series): Series of prices.
    
    Returns:
    - Tuple[float, float]: Buy and Hold, Buy and Hold Signal.
    """
    bandh_returns = []
    buy_price = None

    for index in range(prices.shape[0]):
        if index == 0:
            buy_price = prices.iloc[index]
        elif index == prices.shape[0] - 1 and buy_price is not None:
            bandh_returns.append((prices.iloc[index] - (buy_price * 1.00025) ) / buy_price)
            buy_price = 0
    bandh_returns = np.array(bandh_returns)
    return bandh_returns.mean()

# Define the tickers and download the data
tickers = ["ALL", "ASGN", "CI", "COP", "EME", "EVR", "GILD", "GPK", 
           "ISRG", "MKL", "MOH", "PEG", "PXD", "QCOM", "UBSI", "VFC", "XEL"]

# Download the adjusted close prices for the specified time period
start_date = "2017-11-27"
end_date = "2019-11-27"
data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

rors_macd = []
risks_macd = []
sharpe_ratios_macd = []

rors_rsi = []
risks_rsi = []
sharpe_ratios_rsi = []

rors_bandh = []

# Apply strategies and compute metrics
for ticker in tickers:
    prices = data[ticker]
    macd_return,macd_risk, macd_sharpe = calculate_macd_matrices(prices)
    rsi_return, rsi_risk, rsi_sharpe = calculate_rsi_matrices(prices)
    bandh_return = calculate_bandh_matrices(prices)
    rors_macd.append(macd_return)
    risks_macd.append(macd_risk)
    sharpe_ratios_macd.append(macd_sharpe)

    rors_rsi.append(rsi_return)
    risks_rsi.append(rsi_risk)
    sharpe_ratios_rsi.append(rsi_sharpe)

    rors_bandh.append(bandh_return)

    print(f"Metrics for {ticker}:")
    
    print(f"MACD - Rate of Return: {macd_return:.5f}, Risk: {macd_risk:.5f}, Sharpe Ratio: {macd_sharpe:.5f}")
    print(f"RSI - Rate of Return: {rsi_return:.5f}, Risk: {rsi_risk:.2f}, Sharpe Ratio: {rsi_sharpe:.5f}")
    print(f"Buy and Hold - Rate of Return: {bandh_return:.2f}")

print("-" * 50)
print("Average Metrics:")
print(f"MACD - Rate of Return: {np.array(rors_macd).mean():.5f}, Risk: {np.array(risks_macd).mean():.5f}, Sharpe Ratio: {np.array(sharpe_ratios_macd).mean():.5f}")
print(f"RSI - Rate of Return: {np.array(rors_rsi).mean():.5f}, Risk: {np.array(risks_rsi).mean():.2f}, Sharpe Ratio: {np.array(sharpe_ratios_rsi).mean():.5f}")
print(f"Buy and Hold - Rate of Return: {np.array(rors_bandh).mean():.2f}")

print("-" * 50)
print("Median Metrics:")
print(f"MACD - Rate of Return: {np.median(rors_macd):.5f}, Risk: {np.median(risks_macd):.5f}, Sharpe Ratio: {np.median(sharpe_ratios_macd):.5f}")
print(f"RSI - Rate of Return: {np.median(rors_rsi):.5f}, Risk: {np.median(risks_rsi):.2f}, Sharpe Ratio: {np.median(sharpe_ratios_rsi):.5f}")
print(f"Buy and Hold - Rate of Return: {np.median(rors_bandh):.2f}")

[*********************100%%**********************]  17 of 17 completed
Metrics for ALL:
MACD - Rate of Return: -0.10796, Risk: 0.04580, Sharpe Ratio: -2.90284
RSI - Rate of Return: 0.07599, Risk: 0.01, Sharpe Ratio: 3.68513
Buy and Hold - Rate of Return: 0.18
Metrics for ASGN:
MACD - Rate of Return: 0.25887, Risk: 0.05671, Sharpe Ratio: 4.12407
RSI - Rate of Return: 0.11166, Risk: 0.02, Sharpe Ratio: 4.45513
Buy and Hold - Rate of Return: 0.06
Metrics for CI:
MACD - Rate of Return: -0.39754, Risk: 0.05330, Sharpe Ratio: -7.92740
RSI - Rate of Return: 0.06435, Risk: 0.01, Sharpe Ratio: 3.22092
Buy and Hold - Rate of Return: 0.00
Metrics for COP:
MACD - Rate of Return: 0.23079, Risk: 0.06380, Sharpe Ratio: 3.22576
RSI - Rate of Return: 0.10930, Risk: 0.03, Sharpe Ratio: 2.95379
Buy and Hold - Rate of Return: 0.27
Metrics for EME:
MACD - Rate of Return: 0.06772, Risk: 0.04222, Sharpe Ratio: 1.01191
RSI - Rate of Return: 0.09866, Risk: 0.03, Sharpe Ratio: 2.46449
Buy and Hold - Rate of Ret

In [77]:

def split_func(df):
    # Define the split ratios
    train_ratio = 0.8

    # Calculate the split indices
    total_rows = len(df)
    train_split_idx = int(total_rows * train_ratio)

    # Split the data
    train_df = df.iloc[:train_split_idx].reset_index(drop=True)
    test_df = df.iloc[train_split_idx + 3:].reset_index(drop=True)

    return train_df, test_df


In [83]:
from strategy1 import load_strategy_1, strategy1_fitness_function

# Read the data from CSV
df = pd.read_csv("data/stock_data.csv")

train_df, test_df = split_func(df)
# Define thresholds
main_thresholds = (
    np.array([0.098, 0.22, 0.48, 0.72, 0.98, 1.22, 1.55, 1.70, 2, 2.55])
    / 100
)
for threshold in main_thresholds:
    # Load strategy 1 decisions
    stock_decision_by_threshold = load_strategy_1(
        df=test_df,
        thresholds=[threshold],
        pkl_filename=f"data/strategy1{threshold}_test_data.pkl",
    )

    RoR, volatility, sharpe_ratio = strategy1_fitness_function(
        test_df, [1], stock_decision_by_threshold
    )

    print(
        f"Strategy 1 matrices for threshold {threshold * 100}:"
    )
    print(f"RoR: {RoR:.5f}, Risk: {volatility:.5f}, Sharpe Ratio: {sharpe_ratio:.5f}")

Strategy 1 matrices for threshold 0.098:
RoR: 0.10203, Risk: 0.03671, Sharpe Ratio: 3.08710
Strategy 1 matrices for threshold 0.22:
RoR: 0.17357, Risk: 0.04836, Sharpe Ratio: 2.56889
Strategy 1 matrices for threshold 0.48:
RoR: 0.15082, Risk: 0.04779, Sharpe Ratio: 2.47264
Strategy 1 matrices for threshold 0.72:
RoR: 0.11916, Risk: 0.04844, Sharpe Ratio: 1.77385
Strategy 1 matrices for threshold 0.98:
RoR: 0.11648, Risk: 0.05052, Sharpe Ratio: 1.96221
Strategy 1 matrices for threshold 1.22:
RoR: 0.15452, Risk: 0.04924, Sharpe Ratio: 2.74755
Strategy 1 matrices for threshold 1.55:
RoR: 0.14330, Risk: 0.04938, Sharpe Ratio: 2.36734
Strategy 1 matrices for threshold 1.7000000000000002:
RoR: 0.15894, Risk: 0.05026, Sharpe Ratio: 2.53725
Strategy 1 matrices for threshold 2.0:
RoR: 0.11197, Risk: 0.05240, Sharpe Ratio: 1.67957
Strategy 1 matrices for threshold 2.55:
RoR: 0.11496, Risk: 0.06048, Sharpe Ratio: 2.01260


In [12]:
from strategy2 import load_strategy_2, strategy2_fitness_function

# Read the data from CSV
df = pd.read_csv("data/stock_data.csv")

train_df, test_df = split_func(df)
# Define thresholds
main_thresholds = (
    np.array([0.098, 0.22, 0.48, 0.72, 0.98, 1.22, 1.55, 1.70, 2, 2.55])
    / 100
)
for threshold in main_thresholds:
    # Load strategy 1 decisions
    stock_decision_by_threshold = load_strategy_2(
        df=test_df,
        thresholds=[threshold],
        pkl_filename=f"data/strategy2{threshold}_test_data.pkl",
    )

    RoR, volatility, sharpe_ratio = strategy2_fitness_function(
        test_df, [1], stock_decision_by_threshold
    )

    print(
        f"Strategy 2 matrices for threshold {threshold * 100:.3f}:"
    )
    print(f"RoR: {RoR:.5f}, Risk: {volatility:.5f}, Sharpe Ratio: {sharpe_ratio:.5f}")

Strategy 2 matrices for threshold 0.098:
RoR: 0.14243, Risk: 0.04890, Sharpe Ratio: 1.34334
Strategy 2 matrices for threshold 0.220:
RoR: 0.11255, Risk: 0.04667, Sharpe Ratio: 0.71992
Strategy 2 matrices for threshold 0.480:
RoR: -0.00107, Risk: 0.03695, Sharpe Ratio: -0.41817
Strategy 2 matrices for threshold 0.720:
RoR: -0.00052, Risk: 0.04291, Sharpe Ratio: -1.10278
Strategy 2 matrices for threshold 0.980:
RoR: 0.05090, Risk: 0.04233, Sharpe Ratio: 0.36761
Strategy 2 matrices for threshold 1.220:
RoR: 0.13240, Risk: 0.05496, Sharpe Ratio: 1.48737
Strategy 2 matrices for threshold 1.550:
RoR: 0.12523, Risk: 0.05936, Sharpe Ratio: 0.93892
Strategy 2 matrices for threshold 1.700:
RoR: 0.02561, Risk: 0.04399, Sharpe Ratio: -0.22031
Strategy 2 matrices for threshold 2.000:
RoR: 0.07249, Risk: 0.04059, Sharpe Ratio: 0.97187
Strategy 2 matrices for threshold 2.550:
RoR: 0.03594, Risk: 0.04663, Sharpe Ratio: 0.11205


In [15]:
from strategy3 import load_strategy_3, strategy3_fitness_function

# Read the data from CSV
df = pd.read_csv("data/stock_data.csv")

train_df, test_df = split_func(df)
# Define thresholds
main_thresholds = (
    np.array([0.098, 0.22, 0.48, 0.72, 0.98])
    / 100
)
for threshold in main_thresholds:
    # Load strategy 1 decisions
    stock_decision_by_threshold = load_strategy_3(
        df=test_df,
        thresholds=[threshold],
        pkl_filename=f"data/strategy3{threshold}_test_data.pkl",
    )

    RoR, volatility, sharpe_ratio = strategy3_fitness_function(
        test_df, [1], stock_decision_by_threshold
    )

    print(
        f"Strategy 3 matrices for threshold {threshold * 100:.3f}:"
    )
    print(f"RoR: {RoR:.5f}, Risk: {volatility:.5f}, Sharpe Ratio: {sharpe_ratio:.5f}")

Strategy 3 matrices for threshold 0.098:
RoR: 0.17094, Risk: 0.02156, Sharpe Ratio: 7.10973
Strategy 3 matrices for threshold 0.220:
RoR: 0.20783, Risk: 0.02318, Sharpe Ratio: 7.59440
Strategy 3 matrices for threshold 0.480:
RoR: 0.19812, Risk: 0.02306, Sharpe Ratio: 6.82702
Strategy 3 matrices for threshold 0.720:
RoR: 0.14605, Risk: 0.02218, Sharpe Ratio: 5.44326
Strategy 3 matrices for threshold 0.980:
RoR: 0.09250, Risk: 0.01803, Sharpe Ratio: inf


/Users/sheehabpranto/projects/Dissertation/Code/.env/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/sheehabpranto/projects/Dissertation/Code/strategy3.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe_ratio = (RoR - risk_free_rate) / volatility


In [79]:
# Get the weights for strategy 1


# Initialize an empty list to store the weight arrays
weights_list_1 = []

# Open the text file for reading
with open('../results/02/output/strategy1_train_run.txt', 'r') as file:
    # Iterate over each line in the file
    for line in file:
        # Check if the line contains a weight array (using '[' as an indicator)
        if '[' in line:
            # Extract the weight array from the line
            weights = line.split('[')[1].split(']')[0]
            # Convert the weight array from string format to a list of floats
            weights = list(map(float, weights.split()))
            # Append the weight array to the weights_list
            weights_list_1.append(weights)

# Print the weights_list to verify the results
print(len(weights_list_1))


10


In [32]:
# Get the weights for strategy 2


# Initialize an empty list to store the weight arrays
weights_list_2 = []

# Open the text file for reading
with open('../results/02/output/strategy2_train_run.txt', 'r') as file:
    # Iterate over each line in the file
    for line in file:
        # Check if the line contains a weight array (using '[' as an indicator)
        if '[' in line:
            # Extract the weight array from the line
            weights = line.split('[')[1].split(']')[0]
            # Convert the weight array from string format to a list of floats
            weights = list(map(float, weights.split()))
            # Append the weight array to the weights_list
            weights_list_2.append(weights)

# Print the weights_list to verify the results
print(len(weights_list_2))


10


In [33]:
# Get the weights for strategy 3


# Initialize an empty list to store the weight arrays
weights_list_3 = []

# Open the text file for reading
with open('../results/02/output/strategy3_train_run.txt', 'r') as file:
    # Iterate over each line in the file
    for line in file:
        # Check if the line contains a weight array (using '[' as an indicator)
        if '[' in line:
            # Extract the weight array from the line
            weights = line.split('[')[1].split(']')[0]
            # Convert the weight array from string format to a list of floats
            weights = list(map(float, weights.split()))
            # Append the weight array to the weights_list
            weights_list_3.append(weights)

# Print the weights_list to verify the results
print(len(weights_list_3))


14


In [37]:
for weights in weights_list_1:
    print(weights)

[0.069389731, 0.031833682, 0.021503242, 0.039755076, 0.52338761]
[0.00125639, 0.089399, 0.05322611, 0.06408545, 0.5339022, 0.20037699]
[0.00354277, 0.12821646, 0.02566682, 0.07870205, 0.58246386, 0.07141357]
[0.09616388, 0.06232452, 0.01815474, 0.22603512, 0.5055131, 0.01773969]
[0.00606903, 0.04943902, 0.28982133, 0.532353, 0.00549112, 0.01272495]
[0.11415269, 0.01953852, 0.01887584, 0.29635268, 0.50785416, 0.01432284]
[0.11178413, 0.005101155, 3.5353271e-05, 0.129568, 0.52029037]
[0.00694526, 0.01598553, 0.02531379, 0.12820877, 0.52863884, 0.0757412]
[0.02320035, 0.09081082, 0.05465889, 0.5707643, 0.00464999, 0.01571905]
[0.07944951, 0.2739093, 0.00227982, 0.05086539, 0.50489575, 0.03799244]


In [54]:
weights_list_1_old = []
with open('../CEC/Strategy1/ALL/Results/BestModel.txt', 'r') as file:
    for line in file:
        # Split the line by tabs, skip the "Run" text, and convert the rest to floats
        floats = list(map(float, line.strip().split('\t')[1:]))
        weights_list_1_old.append(floats)

weights_list_1_old

[[0.1491520108181337,
  0.02313798739579737,
  0.012877219706440676,
  0.16414051768336327,
  0.19185846577652674,
  0.22105979766865883,
  0.10260236433944772,
  0.005681431945168682,
  0.10310412441481943,
  0.02638608025164366],
 [0.10952976509043179,
  0.0768468912077285,
  0.05878706167210182,
  0.15010031149023426,
  0.25154324422539287,
  0.2544858370818635,
  0.04366850002466732,
  0.000251599661520628,
  0.04105846036585475,
  0.01372832918020468],
 [0.13621560962536572,
  0.025448023852060155,
  0.02193796959414075,
  0.15835876582851766,
  0.2126788241400479,
  0.21494285777483715,
  0.07976347237693199,
  0.028406147091567482,
  0.09507631298839021,
  0.02717201672814094],
 [0.10205257357596606,
  0.08029318162044081,
  0.05752135811030341,
  0.12135414927417493,
  0.24973744352656332,
  0.25083556253680517,
  0.04820863470532321,
  0.0018670376468891924,
  0.056704335172436775,
  0.03142572383109694],
 [0.10680145764831771,
  0.05028426795443379,
  0.025079492655621514,
  

In [82]:
from strategy1 import load_strategy_1, strategy1_fitness_function

# Read the data from CSV
df = pd.read_csv("data/stock_data.csv")

train_df, test_df = split_func(df)

# Define thresholds
main_thresholds = (
    np.array([0.098, 0.22, 0.48, 0.72, 0.98, 1.22, 1.55, 1.70, 2, 2.55])
    / 100
)

for weights in weights_list_1:
    # Load strategy 1 decisions
    stock_decision_by_threshold = load_strategy_1(
        df=test_df,
        thresholds=main_thresholds,
        pkl_filename=f"data/strategy1_test_data.pkl",
        excel_filename=f"output/strategy1_test_data.xlsx",
        export_excel=True,
    )

    RoR, volatility, sharpe_ratio = strategy1_fitness_function(
        test_df, weights, stock_decision_by_threshold
    )
    print(
        f"Strategy 1 matrices:"
    )
    print(f"RoR: {RoR:.5f}, Risk: {volatility:.5f}, Sharpe Ratio: {sharpe_ratio:.5f}")
    print('-' * 50)

Strategy 1 matrices:
RoR: 0.11648, Risk: 0.05052, Sharpe Ratio: 1.96221
--------------------------------------------------
Strategy 1 matrices:
RoR: 0.11648, Risk: 0.05052, Sharpe Ratio: 1.96221
--------------------------------------------------
Strategy 1 matrices:
RoR: 0.11648, Risk: 0.05052, Sharpe Ratio: 1.96221
--------------------------------------------------
Strategy 1 matrices:
RoR: 0.11648, Risk: 0.05052, Sharpe Ratio: 1.96221
--------------------------------------------------
Strategy 1 matrices:
RoR: 0.11916, Risk: 0.04844, Sharpe Ratio: 1.77385
--------------------------------------------------
Strategy 1 matrices:
RoR: 0.11648, Risk: 0.05052, Sharpe Ratio: 1.96221
--------------------------------------------------
Strategy 1 matrices:
RoR: 0.11648, Risk: 0.05052, Sharpe Ratio: 1.96221
--------------------------------------------------
Strategy 1 matrices:
RoR: 0.11648, Risk: 0.05052, Sharpe Ratio: 1.96221
--------------------------------------------------
Strategy 1 matri

In [63]:
from strategy2 import load_strategy_2, strategy2_fitness_function

# Read the data from CSV
df = pd.read_csv("data/stock_data.csv")

train_df, test_df = split_func(df)
# Define thresholds
main_thresholds = (
    np.array([0.098, 0.22, 0.48, 0.72, 0.98, 1.22, 1.55, 1.70, 2, 2.55])
    / 100
)
for weights in weights_list_2:
    # Load strategy 1 decisions
    stock_decision_by_threshold = load_strategy_2(
        df=test_df,
        thresholds=main_thresholds,
        pkl_filename=f"data/strategy2_test_data.pkl",
    )

    RoR, volatility, sharpe_ratio = strategy2_fitness_function(
        test_df, weights, stock_decision_by_threshold
    )
    print(
        f"Strategy 2 matrices:"
    )
    print(f"RoR: {RoR:.5f}, Risk: {volatility:.5f}, Sharpe Ratio: {sharpe_ratio:.5f}")
    print('-' * 50)

Strategy 2 matrices:
RoR: 0.13240, Risk: 0.05496, Sharpe Ratio: 1.48737
--------------------------------------------------
Strategy 2 matrices:
RoR: 0.13240, Risk: 0.05496, Sharpe Ratio: 1.48737
--------------------------------------------------
Strategy 2 matrices:
RoR: 0.13240, Risk: 0.05496, Sharpe Ratio: 1.48737
--------------------------------------------------
Strategy 2 matrices:
RoR: 0.13240, Risk: 0.05496, Sharpe Ratio: 1.48737
--------------------------------------------------
Strategy 2 matrices:
RoR: 0.13240, Risk: 0.05496, Sharpe Ratio: 1.48737
--------------------------------------------------
Strategy 2 matrices:
RoR: 0.13240, Risk: 0.05496, Sharpe Ratio: 1.48737
--------------------------------------------------
Strategy 2 matrices:
RoR: 0.13240, Risk: 0.05496, Sharpe Ratio: 1.48737
--------------------------------------------------
Strategy 2 matrices:
RoR: 0.13240, Risk: 0.05496, Sharpe Ratio: 1.48737
--------------------------------------------------
Strategy 2 matri

In [64]:
from strategy3 import load_strategy_3, strategy3_fitness_function

# Read the data from CSV
df = pd.read_csv("data/stock_data.csv")

train_df, test_df = split_func(df)
# Define thresholds
main_thresholds = (
    np.array([0.098, 0.22, 0.48, 0.72, 0.98])
    / 100
)
for weights in weights_list_3:
    # Load strategy 1 decisions
    stock_decision_by_threshold = load_strategy_3(
        df=test_df,
        thresholds=main_thresholds,
        pkl_filename=f"data/strategy3_test_data.pkl",
    )

    RoR, volatility, sharpe_ratio = strategy3_fitness_function(
        test_df, weights, stock_decision_by_threshold
    )
    print(
        f"Strategy 3 matrices:"
    )
    print(f"Weights: {weights}")
    print(f"RoR: {RoR:.5f}, Risk: {volatility:.5f}, Sharpe Ratio: {sharpe_ratio:.5f}")
    print('-' * 50)

Strategy 3 matrices:
Weights: [0.5029088, 0.25455278, 0.01597133, 0.18433192, 0.0422351]
RoR: 0.17094, Risk: 0.02156, Sharpe Ratio: 7.10973
--------------------------------------------------
Strategy 3 matrices:
Weights: [0.5256536, 0.02469435, 0.00360488, 0.3451369, 0.1009102]
RoR: 0.17094, Risk: 0.02156, Sharpe Ratio: 7.10973
--------------------------------------------------
Strategy 3 matrices:
Weights: [0.530755, 0.21625312, 0.21352546, 0.02953574, 0.00993064]
RoR: 0.17094, Risk: 0.02156, Sharpe Ratio: 7.10973
--------------------------------------------------
Strategy 3 matrices:
Weights: [0.5005853, 0.0612679, 0.40216812, 0.02484093, 0.01113782]
RoR: 0.17094, Risk: 0.02156, Sharpe Ratio: 7.10973
--------------------------------------------------
Strategy 3 matrices:
Weights: [0.50977224, 0.04847394, 0.15175554, 0.18408029, 0.10591806]
RoR: 0.17094, Risk: 0.02156, Sharpe Ratio: 7.10973
--------------------------------------------------
Strategy 3 matrices:
Weights: [0.5375345, 0.

In [69]:
import random

def weighted_random_choice(choices):
    total_weight = sum(weight for choice, weight in choices.items())
    random_num = random.randint(1, total_weight)
    
    for choice, weight in choices.items():
        random_num -= weight
        if random_num <= 0:
            return choice

# Example usage:
choices = {"A": 5, "B": 3, "C": 2}
print(weighted_random_choice(choices))

C


In [74]:
str(tickers)

"['ALL', 'ASGN', 'CI', 'COP', 'EME', 'EVR', 'GILD', 'GPK', 'ISRG', 'MKL', 'MOH', 'PEG', 'PXD', 'QCOM', 'UBSI', 'VFC', 'XEL']"